## TODO:


- [x] List of universities
- [x] Lists of discplines taught in each university
- [x] ~~The currently used list might be outdated~~.
- [x] **Number of students taught on each program?**
- [ ] Compare no. of organizations with some other source of data
    - [x] Compare to https://www.hse.ru/data/2017/06/29/1171183177/IO%202017.%202.%20Obrazovanie%20i%20rynok%20truda.pdf
        - [x] Filter based on bsc/msc programs and unique INNs yields results consistent with HSE report
- [ ] Compare the three sources suggested by Roman: Rosstat, Министерство науки и высшего образования, Social networks?
- [ ] Filter for PhD programs?
- [x] Rosstat doesn't maintain stats on higher education, instead yielding the link to minobrnauki
- [x] Number of students reported by minobrnauki (VPO-1): both overall and region-wise

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.ion()

In [211]:
import os
import requests
import contextlib
import functools
import itertools
import collections
import pprint

In [73]:
def file_or_nothing(download_fn):
    @functools.wraps(download_fn)
    def wrapped(name, *args, **kwargs):
        cache_path = os.path.join('data', name)
        cached = os.path.exists(cache_path)
        try:
            return download_fn(name, *args, **kwargs)
        except:
            if not cached and os.path.exists(cache_path):
                os.remove(os.path.join(cache_path))
            else:
                print('failed but not cleaning up')
    return wrapped


@file_or_nothing
@contextlib.contextmanager
def remote_binary(name, url=None):
    if not os.path.exists('data'):
        os.mkdir('data')
    cache_path = os.path.join('data', name)
    cached = os.path.exists(cache_path)
    assert cached or url is not None
    if not cached:
        with open(cache_path, 'wb') as f, requests.get(url, stream=True) as r:
            for chunk in r.iter_content(chunk_size=4096):
                f.write(chunk)
    with open(cache_path, mode='rb') as f:
        yield f
        
        
@file_or_nothing
@contextlib.contextmanager
def remote_text(name, enc='utf8', url=None):
    if not os.path.exists('data'):
        os.mkdir('data')
    cache_path = os.path.join('data', name)
    cached = os.path.exists(cache_path)
    assert cached or url is not None
    if not cached:
        with open(cache_path, 'w') as f, requests.get(url, stream=True) as r:
            r.encoding = enc
            for chunk in r.iter_lines(chunk_size=10000, decode_unicode=True):
                f.write(chunk)
                f.write('\n')
    with open(cache_path, mode='r') as f:
        yield f

## List of universities

There are some aggregated datasets available
on open-data related governmental resources.
For instance there are stats provided by [open.data.gov.ru](https://data.gov.ru/?language=en) and [obrnadzor](http://obrnadzor.gov.ru)

* * *

### Реестр подведомственных Рособрнадзору федеральных государственных бюджетных учреждений

In [74]:
# http://obrnadzor.gov.ru/ru/opendata/7701537808-PODVED/
with remote_text(
        'podved_1',
        enc='cp1251',
        url='http://obrnadzor.gov.ru/common/upload/opendata/7701537808-PODVED/data-20150325-structure-20150325.csv') as f:
    podved_1 = pd.read_csv(f)

with remote_text(
    'podved_2',
    enc='cp1251',
    url='http://obrnadzor.gov.ru/common/upload/opendata/7701537808-PODVED/data-20141208-structure-20141208.csv') as f:
    podved_2 = pd.read_csv(f, sep=';')
    

In [89]:
podved_1[['name', 'site']]

,name,site
0,ФГБНУ «Федеральный институт педагогических изм...,new.fipi.ru
1,ФГБУ «Федеральный центр тестирования»,http://www.rustest.ru/
2,ФГБНУ «Главный государственный экспертный цент...,http://www.nic.gov.ru/
3,ФГБУ «Национальное аккредитационное агентство ...,www.nica.ru
4,ФГБУ «Информационно-методический центр анализа»,www.imtsa.ru


* * *

### Реестр организаций, осуществляющих образовательную деятельность по имеющим государственную аккредитацию *образовательным программам*

- Page: http://obrnadzor.gov.ru/ru/opendata/7701537808-RAOO/
- Data format: http://isga.obrnadzor.gov.ru/opendata-structure/data-20160908-structure-20160713.xml
- Data:  http://isga.obrnadzor.gov.ru/accredreestr/opendata/

In [90]:
# http://obrnadzor.gov.ru/ru/opendata/7701537808-RAOO/
! wget -c http://isga.obrnadzor.gov.ru/accredreestr/opendata/ -O open_data.zip

--2019-06-07 07:42:51--  http://isga.obrnadzor.gov.ru/accredreestr/opendata/
Resolving isga.obrnadzor.gov.ru (isga.obrnadzor.gov.ru)... 176.99.141.18
Connecting to isga.obrnadzor.gov.ru (isga.obrnadzor.gov.ru)|176.99.141.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72541151 (69M) [application/zip]
Saving to: ‘open_data.zip’

open_data.zip       100%[===================>]  69.18M   203KB/s    in 10m 11s 

2019-06-07 07:53:05 (116 KB/s) - ‘open_data.zip’ saved [72541151/72541151]



In [93]:
! unzip open_data.zip
! mv data-20190607-structure-20160713.xml data/

Archive:  open_data.zip
  inflating: data-20190607-structure-20160713.xml  


In [105]:
import xml.etree.ElementTree as ET
od_tree = ET.parse('data/data-20190607-structure-20160713.xml') # TODO: incremental read
od_certs = od_tree.getroot().getchildren()[0]

In [115]:
for c in itertools.islice(od_certs.getchildren(), 10):
    print(c.find('EduOrgFullName').text)

Муниципальное общеобразовательное учреждение "Кемецкая средняя общеобразовательная школа"
Муниципальное автономное общеобразовательное учреждение «Каменноозерская основная общеобразовательная школа» 
Муниципальное бюджетное общеобразовательное учреждение 
Нижневерейская средняя школа
Муниципальное казённое общеобразовательное учреждение "Согдиондонская средняя общеобразовательная школа"
Негосударственное образовательное учреждение высшего профессионального образования «Академия МНЭПУ»
муниципальное казенное общеобразовательное учреждение Госселекционная средняя школа Камышинского муниципального района Волгоградской области
Бюджетное муниципальное общеобразовательное учреждение "Ахтубинская средняя общеобразовательная школа"
Негосударственное некоммерческое образовательное учреждение высшего профессионального образования «Гуманитарный институт» (г. Москва)
Муниципальное общеобразовательное учреждение "Средняя общеобразовательная школа п. Алгайский Новоузенского района Саратовской област

In [127]:
websites = (org.find('WebSite').text for org in od_certs.iter('ActualEducationOrganization'))
websites = itertools.islice(websites, 20)
websites = list(websites)
pprint.pprint(websites)

['oo-bologoe.ru',
 'oo-bologoe.ru',
 'b17.uralschool.ru',
 'b17.uralschool.ru',
 'http://shkola-nvereja.ucoz.ru',
 'http://shkola-nvereja.ucoz.ru',
 'wwwДневник.ru',
 'wwwДневник.ru',
 None,
 None,
 None,
 'https://gosschool.jimdo.com',
 'https://gosschool.jimdo.com',
 '30ahtubinskaja-soh.edusite.ru',
 '30ahtubinskaja-soh.edusite.ru',
 None,
 None,
 None,
 'http://alga-sool.my1/ru/',
 'http://alga-sool.my1/ru/']


In [184]:
def extract_fields(x, fields):
    fields = {f: x.find(f) for f in fields}
    fields = {f: v.text if v is not None else None for f, v in fields.items()}
    return fields

def actual_org_gist(tree, fields):
    orgs = od_certs.iter('ActualEducationOrganization')
    orgs = (extract_fields(o, fields) for o in orgs)
    return orgs


FILTER_HIGHER_EDU = lambda x: 'высш' in ''.join(map(str, x.values())).lower()

pd.DataFrame(
    filter(FILTER_HIGHER_EDU,
           itertools.islice(
               actual_org_gist(od_tree,
                               ['FullName', 'TypeName']),
               50)))

,FullName,TypeName
0,Образовательное частное учреждение высшего обр...,Образовательная организация высшего образования
1,Образовательное частное учреждение высшего обр...,Образовательная организация высшего образования
2,Пензенский филиал Образовательного частного уч...,None
3,Негосударственное некоммерческое образовательн...,Образовательная организация высшего образования
4,Негосударственное некоммерческое образовательн...,Образовательная организация высшего образования
5,Негосударственное некоммерческое образовательн...,Образовательная организация высшего образования
6,федеральное государственное казенное военное о...,Образовательная организация высшего образования
7,федеральное государственное казенное военное о...,Образовательная организация высшего образования
8,федеральное государственное казенное военное о...,Образовательная организация высшего образования
9,федеральное государственное казенное военное о...,Образовательная организация высшего образования


In [160]:
def iter_higher_edu():
    higher_edu_certs = od_tree.iter('Certificate')
    def is_higher(x):
        org = x.find('ActualEducationOrganization')
        fields = ['FullName', 'KindName']
        fields = [org.find(f) for f in fields]
        fields = [f.text if hasattr(f, 'text') else '' for f in fields]
        return 'высш' in ' '.join(map(str, fields)).lower()
    higher_edu_certs = filter(is_higher, higher_edu_certs)
    return higher_edu_certs

In [179]:
'With this primitive filter, we get an estimate of {} universities in Russia'.format(
    sum(map(lambda x: 1, iter_higher_edu()), 0))

'With this primitive filter, we get an estimate of 7168 universities in Russia'

In [193]:
def is_accredited(prog):
    field = prog.find('IsAccredited')
    return (
        field is not None
        and field.text is not None
        and field.text.strip() == '0')

orgs = iter_higher_edu()
supls = itertools.chain.from_iterable((org.iter('Supplement') for org in orgs))
progs = itertools.chain.from_iterable((supl.iter('EducationalProgram') for supl in supls))
progs = filter(is_accredited, progs)
progs = itertools.islice(progs, 0, 100, 3)


fields = ['ProgrammName', 'EduLevelName', 'TypeName', 'Qualification']
progs = ({f: prog.find(f).text for f in fields} for prog in progs)
pd.DataFrame(progs)

,EduLevelName,ProgrammName,Qualification,TypeName
0,Высшее образование - специалитет,Менеджмент организации,Менеджер,None
1,Среднее профессиональное образование,Экономика и бухгалтерский учет (по отраслям),Бухгалтер,None
2,Послевузовское профессиональное образование,Экономика и управление народным хозяйством (по...,Кандидат наук,послевузовское профессиональное образование (а...
3,Высшее образование - специалитет,Природопользование,Эколог-природопользователь,None
4,Высшее образование - бакалавриат,Юриспруденция,Бакалавр юриспруденции,None
5,Высшее образование - бакалавриат,Естественные науки\n,None,None
6,Высшее образование - бакалавриат,Экономика,Бакалавр экономики,None
7,Высшее образование - специалитет,Юриспруденция,Юрист,None
8,Высшее образование - специалитет,Менеджмент,None,ВО - специалитет
9,Не определен,Антикризисное управление,None,-


## Meaningful `is_higher` filter

Naive filtering based on name of the organization yields the estimate
of about 7k higher education organizations, HSE reports about 1k BSc, 1-2k phd programs, and 2-3k special programs

![](hse_higher.png)
![](hse_prof.png)

In [228]:
def field_hist(tree, section, field):
    orgs = tree.iter(section)
    vals = (extract_fields(o, [field]) for o in orgs)
    vals = collections.Counter((o[field] for o in vals))
    return vals

def hist_msg(field, hist, max_rows=10):
    return '\n'.join([
        'The field "{}" has {} unique values. A sample:'.format(field, len(hist)),
        pprint.pformat(list(itertools.islice(hist.items(), max_rows)))])

def show_hist(tree, section, field, max_rows=10):
    hist = field_hist(tree, section, field)
    print(hist_msg(field, hist))

In [229]:
show_hist(od_tree, 'ActualEducationOrganization', 'KindName', 20)

The field "KindName" has 13 unique values. A sample:
[(None, 159376),
 ('Академия', 3004),
 ('Институт', 4409),
 ('Центр социально-трудовой адаптации и профориентации', 18853),
 ('Университет', 10148),
 ('Колледж', 97),
 ('Техникум', 158),
 ('Научно-исследовательский институт', 130),
 ('Военная академия', 9),
 ('Проектный институт', 2)]


In [230]:
show_hist(od_tree, 'ActualEducationOrganization', 'TypeName', 20)

The field "TypeName" has 17 unique values. A sample:
[(None, 159484),
 ('Образовательная организация высшего образования', 21074),
 ('Образовательное учреждение высшего профессионального образования', 3831),
 ('Научная организация', 18),
 ('Образовательное учреждение среднего профессионального образования', 5232),
 ('Профессиональная образовательная организация', 4305),
 ('Кадетская школа', 170),
 ('Учреждение дополнительного образования взрослых', 12),
 ('Организации дополнительного профессионального образования', 264),
 ('Научные организации', 1456)]


In [232]:
show_hist(od_tree, 'EducationalProgram', 'TypeName', 50)
show_hist(od_tree, 'EducationalProgram', 'EduLevelName', 50)

The field "TypeName" has 29 unique values. A sample:
[(None, 336537),
 ('послевузовское профессиональное образование (аспирантура, адъюнктура)',
  1062),
 ('ВО - специалитет', 61425),
 ('-', 26650),
 ('высшее профессиональное образование', 73375),
 ('ВО - бакалавриат', 28714),
 ('СПО - среднее звено (базовая подготовка)', 906),
 ('СПО - подготовка специалистов среднего звена', 7461),
 ('профессиональная переподготовка', 3584),
 ('среднее (полное) общее образование', 186)]
The field "EduLevelName" has 35 unique values. A sample:
[('Начальное общее образование', 91208),
 ('Среднее общее образование', 67441),
 ('Основное общее образование', 86871),
 ('Высшее образование - специалитет', 66366),
 ('Послевузовское профессиональное образование', 6524),
 ('Среднее профессиональное образование', 77461),
 ('Высшее образование - бакалавриат', 54811),
 ('Не определен', 75251),
 ('Высшее образование - магистратура', 30830),
 ('Высшее профессиональное образование', 44208)]


In [218]:
higher_type = 'Образовательная организация высшего образования'
orgs = od_tree.iter('ActualEducationOrganization')
orgs = filter(lambda org: getattr(org.find('TypeName'), 'text', None) == higher_type, orgs)
orgs_no = sum(map(lambda x: 1, orgs), 0)
print('No. of "{}": {}'.format(higher_type, orgs_no))

No. of "Образовательная организация высшего образования": 21074


In [222]:
def has_higher_edu_programs(cert):
    return any((
        (getattr(prog.find('EduLevelName'), 'text', '') or '')
        .startswith('Высшее образование')
        and prog.find('IsAccredited').text == '0'
        for prog in
        cert.iter('EducationalProgram')))

    
higher_type = 'Образовательная организация высшего образования'
certs = od_tree.iter('Certificate')
certs = filter(has_higher_edu_programs, certs)
n_hp_certs = sum(map(lambda x: 1, certs), 0)
print('Number of certificates referring to programs whose titles begin with "Высшее образование": {}'.format(n_hp_certs))

Number of certificates referring to programs whose titles begin with "Высшее образование": 6752


In [242]:
def contains_any(s: str, substrings: list):
    return any((ss in s for ss in substrings))

def has_levels(cert, levels):
    return any((
        contains_any((getattr(prog.find('EduLevelName'), 'text', '') or ''),
                     levels)
        and prog.find('IsAccredited').text == '0'
        for prog in
        cert.iter('EducationalProgram')))

    
higher_type = 'Образовательная организация высшего образования'
certs = od_tree.iter('Certificate')
certs = filter(lambda cert: has_levels(cert, ['бакалавриат', 'магистратура']),
               certs)
n_hp_certs = sum(map(lambda x: 1, certs), 0)
print('Number of certificates referring to programs whose titles begin with "Высшее образование": {}'.format(n_hp_certs))

Number of certificates referring to programs whose titles begin with "Высшее образование": 5156


In [244]:
certs = od_tree.iter('Certificate')
certs = filter(lambda cert: has_levels(cert, ['бакалавриат', 'магистратура']),
               certs)
names = collections.Counter(map(lambda c: c.find('EduOrgFullName').text, certs))
print('{} uniq fullnames'.format(len(names)))

2082 uniq fullnames


In [245]:
certs = od_tree.iter('Certificate')
certs = filter(lambda cert: has_levels(cert, ['бакалавриат', 'магистратура']),
               certs)
names = collections.Counter(map(lambda c: c.find('EduOrgINN').text, certs))
print('{} uniq INNs'.format(len(names)))

1174 uniq INNs


In [246]:
certs = od_tree.iter('Certificate')
certs = filter(lambda cert: has_levels(cert, ['аспирантура']),
               certs)
names = collections.Counter(map(lambda c: c.find('EduOrgINN').text, certs))
print('{} uniq INNs with phd programs'.format(len(names)))

0 uniq INNs with phd programs


### PhD programs?

No such EduLevel

## Number of students

That seems to be a quite more complicated question.
There's no such stats in RAOO.
We could opt to process each uni individually,
but that'd be too much routine work which would need
to be regularly repeated.

Alternative entry point: [graduates employment monitoring](http://vo.graduate.edu.ru/#/?year=2015&year_monitoring=2016).
That page says each educational organization must
report about issued qualification certificates in "Федеральный реестр документов об образовании и (или) квалификации".

**UPD**: http://isga.obrnadzor.gov.ru/accredreestr/

In [192]:
orgs = iter_higher_edu()
extractor = functools.partial(extract_fields, fields=['EduOrgFullName', 'RegNumber', 'SerialNumber', 'FormNumber'])
orgs = map(extractor, orgs)
orgs = itertools.islice(orgs, 10)
pprint.pprint(list(orgs))

[{'EduOrgFullName': 'Негосударственное образовательное учреждение высшего '
                    'профессионального образования «Академия МНЭПУ»',
  'FormNumber': '0000815',
  'RegNumber': '0757',
  'SerialNumber': '90А01'},
 {'EduOrgFullName': 'Негосударственное некоммерческое образовательное '
                    'учреждение высшего профессионального образования '
                    '«Гуманитарный институт» (г. Москва)',
  'FormNumber': '001340',
  'RegNumber': '1286',
  'SerialNumber': 'A'},
 {'EduOrgFullName': 'федеральное государственное казенное военное '
                    'образовательное учреждение высшего образования «Тюменское '
                    'высшее военно-инженерное командное училище (военный '
                    'институт) имени маршала инженерных войск А.И.Прошлякова» '
                    'Министерства обороны Российской Федерации',
  'FormNumber': '0001928',
  'RegNumber': '1834',
  'SerialNumber': '90А01'},
 {'EduOrgFullName': 'федеральное государственное бюдж

### Minobr data, VPO-1 forms:
https://minobrnauki.gov.ru/ru/activity/statan/stat/highed/

In [247]:
! wget -c https://minobrnauki.gov.ru/common/upload/download/VPO_1_2018.rar
! 7z x VPO_1_2018.rar

--2019-06-07 12:47:54--  https://minobrnauki.gov.ru/common/upload/download/VPO_1_2018.rar
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving minobrnauki.gov.ru (minobrnauki.gov.ru)... 217.107.75.116
Connecting to minobrnauki.gov.ru (minobrnauki.gov.ru)|217.107.75.116|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz (806EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 26075805 bytes (25 MiB)

Extracting archive: VPO_1_2018.rar
 15% 100 Ope            --
Path = VPO_1_2018.rar
Type = Rar
Physical Size = 26075805
Solid = -
Blocks = 731
Multivolume = -
Volumes = 1

     18% 131 - Своды ВПО-1 2018/Го . тация экстернов.x                                             

# Validation

## HSE study

<img src="./Screenshot_2019-06-07 InObraz_KNIGA_17 indb - IO 2017 2 Obrazovanie i rynok truda pdf.png"/>

- https://www.hse.ru/data/2017/06/29/1171183177/IO%202017.%202.%20Obrazovanie%20i%20rynok%20truda.pdf
- https://www.hse.ru/org/hse/primarydata/